In [1]:
#%matplotlib inline
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
from siamese.dataset import Dataset
from siamese.dataset import DataLoader
from siamese.model import *
from extensies import metrics as mt
from extensies import preprocessing as ps
from matplotlib import gridspec

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
import pandas as pd 
import ast

columns = ['date','name','modality','size','model','num_of_data','iterations','augmentation',
           'normalization','acc','acc_w','acc_p','auc','auc_w','auc_p','komentar']

df = pd.read_csv('records.csv',index_col=0)

In [10]:
df

,acc,acc_p,acc_w,auc,auc_p,auc_w,augmentation,date,iterations,kfold_id,komentar,modality,model,name,normalization,num_of_data,size
0,0.744898,0.673469,0.265306,0.508658,0.532468,0.515152,False,2019-03-20,1000,NaN,NaN,['t2tsetra/t/40x40x1/'],siamese,t2tsetra_40x40x1_30090,ScaleNormalization,323,40x40x1
1,0.795918,0.724490,0.295918,0.575758,0.564935,0.551948,False,2019-03-20,2000,NaN,NaN,['t2tsetra/t/40x40x1/'],siamese,t2tsetra_40x40x1_24089,ScaleNormalization,323,40x40x1
2,0.806122,0.755102,0.387755,0.651515,0.740260,0.593074,False,2019-03-20,3000,NaN,NaN,['t2tsetra/t/40x40x1/'],siamese,t2tsetra_40x40x1_51626,ScaleNormalization,323,40x40x1
3,0.755102,0.683673,0.397959,0.549784,0.642857,0.599567,False,2019-03-20,4000,NaN,NaN,['t2tsetra/t/40x40x1/'],siamese,t2tsetra_40x40x1_96379,ScaleNormalization,323,40x40x1
4,0.775510,0.704082,0.397959,0.597403,0.673160,0.547619,False,2019-03-20,1000,NaN,NaN,['adc/t/40x40x1/'],siamese,adc_40x40x1_72631,ScaleNormalization,324,40x40x1
5,0.795918,0.775510,0.540816,0.662338,0.701299,0.638528,False,2019-03-20,2000,NaN,NaN,['adc/t/40x40x1/'],siamese,adc_40x40x1_18731,ScaleNormalization,324,40x40x1
6,0.704082,0.683673,0.510204,0.621212,0.660173,0.584416,False,2019-03-20,3000,NaN,NaN,['adc/t/40x40x1/'],siamese,adc_40x40x1_86012,ScaleNormalization,324,40x40x1
7,0.795918,0.724490,0.285714,0.541126,0.616883,0.545455,Elastic,2019-03-20,1000,NaN,NaN,"['adc/t/40x40x1/', 't2tsetra/t/40x40x1']",siamese,t2tsetra_adc_40x40x1_36217,ScaleNormalization,4098,40x40x1
8,0.775510,0.714286,0.255102,0.510823,0.575758,0.508658,Elastic,2019-03-20,2000,NaN,NaN,"['adc/t/40x40x1/', 't2tsetra/t/40x40x1']",siamese,t2tsetra_adc_40x40x1_47376,ScaleNormalization,4098,40x40x1
9,0.795918,0.744898,0.255102,0.523810,0.595238,0.491342,Elastic,2019-03-20,2500,NaN,NaN,"['adc/t/40x40x1/', 't2tsetra/t/40x40x1']",siamese,t2tsetra_adc_40x40x1_62224,ScaleNormalization,4098,40x40x1


In [23]:
from functools import reduce 
a = {'a': [1,1,3],'b':[2,3,4],'c': [3,4,5]}
reduce(lambda x, value:x + value[1], a.values(), 0)
len(a)


3

In [25]:
print("{0:.4f}".format(0.5556554))

0.5557


In [8]:
idx = 45

model_name = df.iloc[idx]['name']
modalities = df.iloc[idx]['modality']
modalities = ast.literal_eval(modalities)
combined = True

In [9]:
model_name

'adc_20x20x1_98058'

In [10]:
if combined:
    loader = DataLoader('../../data/',modalities)
    loader.load_data()
    X,y = loader.combine_channels(modalities)
    X_train, X_test, y_train, y_test = loader.get_train_test('combined')
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

else:
    loader = DataLoader('../../data/',modalities)
    loader.load_data()
    X,y = loader.get_data(modalities[0])
    X_train, X_test, y_train, y_test = loader.get_train_test(modalities[0])
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
X_train[0].shape

(40, 40, 3)

### Simple modality

In [10]:
dataset = Dataset()

In [11]:
dataset.images_train = X_train
dataset.images_test = X_test
dataset.labels_train = y_train
dataset.labels_test = y_test


## Create the siamese net feature extraction model

In [12]:
img_placeholder = tf.placeholder(tf.float32, [None] + list(dataset.images_train.shape[1:]), name='img')
net = mnist_model(img_placeholder, reuse=False)

## Restore from checkpoint and calc the features from all of train data

In [13]:
#run the train image through the network to get the test features
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ckpt = tf.train.get_checkpoint_state("model")
    saver.restore(sess, "model/" + model_name + ".ckpt")
    train_feat = sess.run(net, feed_dict={img_placeholder:dataset.images_train})

INFO:tensorflow:Restoring parameters from model/ktrans_a_40x40x1_82320.ckpt


In [14]:
#run the test image through the network to get the test features
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ckpt = tf.train.get_checkpoint_state("model")
    saver.restore(sess, "model/" + model_name + ".ckpt")
    search_feat = sess.run(net, feed_dict={img_placeholder:dataset.images_test})
    

INFO:tensorflow:Restoring parameters from model/ktrans_a_40x40x1_82320.ckpt


## Count similarity

In [55]:
y_pred = []
y_pred_t = []
y_pred_w = []
for idx,feat in enumerate(search_feat):
    #calculate the cosine similarity and sort
    y_pred.append(mt.siamese_predict(train_feat,feat,dataset))
    y_pred_t.append(mt.treshold_predict(train_feat,feat,dataset,0.4,10,distance= 'euclidean'))
    y_pred_w.append(mt.weighted_predict(train_feat,feat,dataset,0.4,10))
    

In [56]:
accuracy_score(dataset.labels_test,y_pred)

0.7525773195876289

In [57]:
accuracy_score(dataset.labels_test,y_pred_t)

0.7525773195876289

In [58]:
accuracy_score(dataset.labels_test,y_pred_w)

0.5051546391752577

In [59]:
fpr, tpr, thresholds = metrics.roc_curve(dataset.labels_test, y_pred, pos_label=1)
metrics.auc(fpr, tpr)

0.5839041095890412

In [60]:
fpr, tpr, thresholds = metrics.roc_curve(dataset.labels_test, y_pred_t, pos_label=1)
metrics.auc(fpr, tpr)

0.6678082191780821

In [61]:
fpr, tpr, thresholds = metrics.roc_curve(dataset.labels_test, y_pred_w, pos_label=1)
metrics.auc(fpr, tpr)

0.6292808219178082

In [45]:
print(classification_report(dataset.labels_test,y_pred_t))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83        73
           1       0.45      0.38      0.41        24

   micro avg       0.73      0.73      0.73        97
   macro avg       0.63      0.61      0.62        97
weighted avg       0.72      0.73      0.72        97

